<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/GluonTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mxnet gluonts

In [ ]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx import Trainer
from gluonts.dataset.common import ListDataset
import pandas as pd
import matplotlib.pyplot as plt
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions

# Load the dataset
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv')
df.columns = ['Month', 'Passengers']

# Prepare the training data
training_data = ListDataset([{"start": df['Month'][0], "target": df['Passengers'].values}], freq="M")

# Initialize the estimator
estimator = DeepAREstimator(freq="M", prediction_length=12, trainer=Trainer(epochs=5))

# Train the model
predictor = estimator.train(training_data)

# Create a dataset for prediction
test_data = ListDataset([{"start": df['Month'][0], "target": df['Passengers'].values}], freq="M")

# Make predictions
forecast_it, ts_it = make_evaluation_predictions(test_data, predictor=predictor, num_samples=100)

# Convert the forecast iterator to a list of forecasts
forecasts = list(forecast_it)
tss = list(ts_it)

# Plot the predictions
def plot_forecast(tss, forecasts):
    for target, forecast in zip(tss, forecasts):
        target.plot(color="b")
        forecast.plot(color="g", prediction_intervals=(50.0, 90.0))
    plt.show()

plot_forecast(tss, forecasts)